In [30]:
exit(0)

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
!pip install transformers -q

In [10]:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 149.9 MB 26 kB/s 
     |████████████████████████████████| 831.4 MB 2.3 kB/s 
     |████████████████████████████████| 57 kB 4.6 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.9

In [11]:
!pip install --upgrade pandas==1.3.4

     |████████████████████████████████| 11.3 MB 5.3 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.4 which is incompatible.


In [1]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

In [2]:
dev = xm.xla_device()
dev

device(type='xla', index=1)

In [3]:
import glob
import os
import pandas as pd

def load_tweets(base_path, columns=None):
    if columns is None:
        columns = ['id', 'time', 'retweet_count', 'content', 'player']

    tweets_batch = []
    for file in glob.glob(base_path):
        current_data = pd.read_csv(file, sep=";")
        current_data['player'] = file.split(os.path.sep)[-1].split(".csv")[0]
        tweets_batch.append(current_data)

    data = pd.concat(tweets_batch)
    data = data[columns]
    return data


In [4]:
from tqdm.notebook import trange, tqdm

In [ ]:
!rm -rf ./cardiffnlp

In [6]:
from scipy.special import softmax

In [8]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import pickle


def normalize_score(negative, neutral, positive):
    return ((positive + neutral / 2) * 2) - 1 if positive > negative else -((negative + neutral / 2) * 2) + 1


def preprocess(text):
    new_text = []

    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


tweets = load_tweets("tweets/*.csv")

task = 'sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(dev)


In [10]:
scores_map = {}


In [11]:
batch_size = 1024
batches = np.array_split(tweets, len(tweets) // batch_size)

In [ ]:
for batch in tqdm(batches):
    ids = batch['id'].values
    texts = batch['content'].map(lambda x: preprocess(x)).values

    if ids[-1] in scores_map:
      continue

    encoded_input =  tokenizer(list(texts),return_tensors='pt',padding=True,truncation=True, max_length=128)
    data_input_ids = encoded_input['input_ids'].to(dev)
    data_attention_mask = encoded_input['attention_mask'].to(dev)

    output = model.forward(input_ids=data_input_ids,attention_mask=data_attention_mask)

    scores = softmax(output.logits.cpu().detach().numpy(), axis=1)

    for idx, score in enumerate(scores):
        scores_map[ids[idx]] = normalize_score(*score)

with open('scores.pkl', 'wb') as file:
    pickle.dump(scores_map, file, pickle.HIGHEST_PROTOCOL)


  0%|          | 0/222 [00:00<?, ?it/s]